# Introduction

As part of the [ Google Data Analytics Professional Certificate](http://www.coursera.org/professional-certificates/google-data-analytics), I am excited to share my approach to the Cyclistic bike-share analysis case study. In this engaging case study, I am playing the role of a junior data analyst within the marketing team at Cyclistic, a fictional bike-share company based in Chicago, collaborating closely with key team members to tackle the business questions at hand and deliver insightful results.

To carry out the project successfully, I follow a well-defined set of steps. Google's certificate program outlines the data analysis process, which includes the following stages: asking the right questions, preparing the data, processing it effectively, analyzing the findings, sharing the results, and taking appropriate actions. Within this notebook, I present a practical demonstration of how I apply these steps to analyze the given dataset.

I decided to use SQLite for this analysis to push my SQL skills further. But I also recreated the whole project in R for an extra challenge.[You can see it here.](https://www.kaggle.com/code/esinsari/cyclistic-bike-share-case-study-with-r)

In order to identify trends, I will be closely examining the Cyclistic historical bike trip data over a 12-month period, starting from March 2023 and ending in February 2024. Motivate International Inc. has provided the data under this [license](http://www.divvybikes.com/data-license-agreement) for public access. The datasets, which are in the form of 'csv' files, can be found [here](http://divvy-tripdata.s3.amazonaws.com/index.html).

First, I started by importing the dataset into DBeaver. I made sure the table structure and data types matched the original project in R. Then, I recreated the SQL queries and data transformations that I had previously implemented in R


In [1]:
--- Combine data from all monthly tables into one

CREATE TABLE tripdata AS
SELECT * FROM jan24
UNION ALL
SELECT * FROM feb24
UNION ALL
SELECT * FROM mar23
UNION ALL
SELECT * FROM apr23 
UNION ALL
SELECT * FROM may23 
UNION ALL
SELECT * FROM jun23 
UNION ALL
SELECT * FROM jul23 
UNION ALL
SELECT * FROM aug23 
UNION ALL
SELECT * FROM sep23 
UNION ALL
SELECT * FROM oct23 
UNION ALL
SELECT * FROM nov23 
UNION ALL
SELECT * FROM dec23 

ERROR: Error in parse(text = x, srcfile = src): <text>:1:13: unexpected symbol
1: --- Combine data
                ^


In [ ]:
--- Lets see the structure of the newly merged data

PRAGMA table_info(tripdata);

SELECT COUNT(*) FROM tripdata;

|cid|name|type|notnull|dflt_value|pk|
|---|----|----|-------|----------|--|
|0|ride_id|TEXT|0||0|
|1|rideable_type|TEXT|0||0|
|2|started_at|TEXT|0||0|
|3|ended_at|TEXT|0||0|
|4|start_station_name|TEXT|0||0|
|5|start_station_id||0||0|
|6|end_station_name|TEXT|0||0|
|7|end_station_id||0||0|
|8|start_lat|REAL|0||0|
|9|start_lng|REAL|0||0|
|10|end_lat|REAL|0||0|
|11|end_lng|REAL|0||0|
|12|member_casual|TEXT|0||0|



* I noticed **the start_station_id** and **end_station_id** columns had N/A values, which aren't ideal. Since SQLite doesn't allow modify column types, I created a new table with the right data types (TEXT in this case) and transferred the data to the new table.


In [ ]:
CREATE TABLE tripdata2 (
    ride_id TEXT,
    rideable_type TEXT,
    started_at TEXT,
    ended_at TEXT,
    start_station_name TEXT,
    start_station_id TEXT,
    end_station_name TEXT,
    end_station_id TEXT,
    start_lat REAL,
    start_lng REAL,
    end_lat REAL,
    end_lng REAL,
    member_casual TEXT
    );

INSERT INTO tripdata2 SELECT * FROM tripdata;

DROP TABLE tripdata;

ALTER TABLE tripdata2 RENAME TO tripdata;


In [ ]:
--- Check for duplicate values in the "ride_id" column 


SELECT COUNT(DISTINCT ride_id) AS distinct_count, COUNT(*) AS total_count
FROM tripdata;

|distinct_count|total_count|
|--------------|-----------|
|5707168|5707168|


In [ ]:
--- Add columns that list the date, month, day, and year of each ride

--- Add new columns with TEXT and DATE data types

ALTER TABLE tripdata
ADD COLUMN start_date DATE;

ALTER TABLE tripdata
ADD COLUMN start_month TEXT;

ALTER TABLE tripdata
ADD COLUMN start_day TEXT;

ALTER TABLE tripdata
ADD COLUMN start_year TEXT;

ALTER TABLE tripdata
ADD COLUMN start_day_of_week TEXT;

In [ ]:
-- Update the new columns with formatted date values
UPDATE tripdata
SET start_date = strftime('%Y-%m-%d', started_at),
    start_day = strftime('%d', started_at),
    start_year = strftime('%Y', started_at);   


In [ ]:
-- Update the start_month column with the month abbreviation
UPDATE tripdata
SET start_month = CASE 
                    WHEN substr(started_at, 6, 2) = '01' THEN 'Jan'
                    WHEN substr(started_at, 6, 2) = '02' THEN 'Feb'
                    WHEN substr(started_at, 6, 2) = '03' THEN 'Mar'
                    WHEN substr(started_at, 6, 2) = '04' THEN 'Apr'
                    WHEN substr(started_at, 6, 2) = '05' THEN 'May'
                    WHEN substr(started_at, 6, 2) = '06' THEN 'Jun'
                    WHEN substr(started_at, 6, 2) = '07' THEN 'Jul'
                    WHEN substr(started_at, 6, 2) = '08' THEN 'Aug'
                    WHEN substr(started_at, 6, 2) = '09' THEN 'Sep'
                    WHEN substr(started_at, 6, 2) = '10' THEN 'Oct'
                    WHEN substr(started_at, 6, 2) = '11' THEN 'Nov'
                    ELSE 'Dec'
                 END;

In [ ]:
-- Update the start_day_of_week column with the day of the week

UPDATE tripdata
SET start_day_of_week = CASE 
                            WHEN started_at IS NULL THEN NULL 
                            WHEN strftime('%w', started_at) = '0' THEN 'Sunday'
                            WHEN strftime('%w', started_at) = '1' THEN 'Monday'
                            WHEN strftime('%w', started_at) = '2' THEN 'Tuesday'
                            WHEN strftime('%w', started_at) = '3' THEN 'Wednesday'
                            WHEN strftime('%w', started_at) = '4' THEN 'Thursday'
                            WHEN strftime('%w', started_at) = '5' THEN 'Friday'
                            ELSE 'Saturday'
                        END;

                       

In [ ]:
-- Add ride_length column and calculate ride_length in minutes
                      
ALTER TABLE tripdata ADD COLUMN ride_length REAL;  


UPDATE tripdata
SET ride_length = ROUND((julianday(ended_at) - julianday(started_at)) * 24 * 60, 2);


In [ ]:
---  verify the data, including the newly added columns and any other modifications I've made

SELECT * FROM tripdata LIMIT 10;

|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|start_date|start_month|start_day|start_year|start_day_of_week|ride_length|
|-------|-------------|----------|--------|------------------|----------------|----------------|--------------|---------|---------|-------|-------|-------------|----------|-----------|---------|----------|-----------------|-----------|
|C1D650626C8C899A|electric_bike|2024-01-12 15:30:27|2024-01-12 15:37:59|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.903267384|-87.63473677600001|41.88917683258|-87.6385057718|member|2024-01-12|Jan|12|2024|Friday|7.529999999999999|
|EECD38BDB25BFCB0|electric_bike|2024-01-08 15:45:46|2024-01-08 15:52:59|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.902936499999996|-87.63444016666666|41.88917683258|-87.6385057718|member|2024-01-08|Jan|08|2024|Monday|7.220000000000001|
|F4A9CE78061F17F7|electric_bike|2024-01-27 12:27:19|2024-01-27 12:35:19|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.90295133333333|-87.63447033333334|41.88917683258|-87.6385057718|member|2024-01-27|Jan|27|2024|Saturday|8.0|
|0A0D9E15EE50B171|classic_bike|2024-01-29 16:26:17|2024-01-29 16:56:06|Wells St & Randolph St|TA1305000030|Larrabee St & Webster Ave|13193|41.884295|-87.63396300000002|41.921822000000006|-87.64414000000001|member|2024-01-29|Jan|29|2024|Monday|29.82|
|33FFC9805E3EFF9A|classic_bike|2024-01-31 05:43:23|2024-01-31 06:09:35|Lincoln Ave & Waveland Ave|13253|Kingsbury St & Kinzie St|KA1503000043|41.948797|-87.675278|41.88917683258|-87.6385057718|member|2024-01-31|Jan|31|2024|Wednesday|26.2|
|C96080812CD285C5|classic_bike|2024-01-07 11:21:24|2024-01-07 11:30:03|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.90322200000001|-87.63432399999999|41.88917683258|-87.6385057718|member|2024-01-07|Jan|07|2024|Sunday|8.65|
|0EA7CB313D4F456A|classic_bike|2024-01-05 14:44:12|2024-01-05 14:53:06|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.90322200000001|-87.63432399999999|41.88917683258|-87.6385057718|member|2024-01-05|Jan|05|2024|Friday|8.9|
|EE11F3A3B39CFBD8|electric_bike|2024-01-04 18:19:53|2024-01-04 18:28:04|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.90336811499999|-87.63486135|41.88917683258|-87.6385057718|member|2024-01-04|Jan|04|2024|Thursday|8.18|
|63E83DE8E3279F15|classic_bike|2024-01-01 14:46:53|2024-01-01 14:57:02|Wells St & Elm St|KA1504000135|Kingsbury St & Kinzie St|KA1503000043|41.90322200000001|-87.63432399999999|41.88917683258|-87.6385057718|member|2024-01-01|Jan|01|2024|Monday|10.15|
|8005682869122D93|electric_bike|2024-01-03 19:31:08|2024-01-03 19:40:05|Clark St & Ida B Wells Dr|TA1305000009|Kingsbury St & Kinzie St|KA1503000043|41.876033500000005|-87.630866|41.88917683258|-87.6385057718|member|2024-01-03|Jan|03|2024|Wednesday|8.95|


In [ ]:
/* 
* Remove rows FROM the tripdata table where the start_station_name is "HQ QR" or the ride_length is less than or equal to 0
***** NOTE !!! Back up your data if you want to avoid permanent data loss
*/

DELETE FROM tripdata WHERE start_station_name = 'HQ QR' OR ride_length <= 0;

In [ ]:
--  Calculate the minimum, MAXimum, average values of the ride_length column

SELECT
    MIN(ride_length) AS min_ride_length,
    MAX(ride_length) AS MAX_ride_length,
    AVG(ride_length) AS avg_ride_length
FROM tripdata;


|min_ride_length|MAX_ride_length|avg_ride_length|
|---------------|---------------|---------------|
|0.02|98489.06999999999|18.33351781465149|


In [ ]:
SELECT member_casual,
    MIN(ride_length) AS min_ride_length,
    MAX(ride_length) AS MAX_ride_length,
    AVG(ride_length) AS avg_ride_length
FROM tripdata 
GROUP BY member_casual 

|member_casual|min_ride_length|MAX_ride_length|avg_ride_length|
|-------------|---------------|---------------|---------------|
|casual|0.02|98489.06999999999|28.311242518458094|
|member|0.02|1559.67|12.750348870708166|


In [ ]:
-- The Maximum ride length for casual riders is exceptionally high at 98,489 minutes (about 68 days)
-- Both casual riders and members have a minimum ride length of 0.02 minutes, which requires correction
 
-- Lets remove rows FROM the tripdata table where ride_length is either greater than or equal to 1440 
(rides longer than a day) or less than 2 (rides less than 2 minutes)

-- NOTE !!! Back up your data if you want to avoid permanent data loss


DELETE FROM tripdata
WHERE ride_length >= 1440 OR ride_length < 2;

In [ ]:
-- We know the user base consists of members and casual riders and different types of bikes are offered 
to users. Lets see if our data backs this up


SELECT DISTINCT member_casual FROM tripdata;

SELECT DISTINCT rideable_type FROM tripdata;

|rideable_type|             |member_casual|  
|-------------|             |-------------|
|electric_bike|             |member|
|classic_bike|              |casual|
|docked_bike|







In [ ]:
--The use of bike types between members and casual riders

SELECT member_casual as User_Type, rideable_type as Bike_Type, COUNT(rideable_type) as Counts
FROM tripdata 
GROUP BY member_casual, rideable_type 


|User_Type|Bike_Type|Counts|
|---------|---------|------|
|casual|classic_bike|860050|
|casual|docked_bike|71939|
|casual|electric_bike|1028217|
|member|classic_bike|1778631|
|member|electric_bike|1703228|


In [ ]:
-- Analyze ridership data by each day for members vs casual riders
    
SELECT member_casual, start_day_of_week, COUNT(*) as number_of_rides,
       ROUND(AVG(ride_length),2) as average_duration
FROM tripdata 
GROUP BY member_casual, start_day_of_week ;  

|member_casual|start_day_of_week|number_of_rides|average_duration|
|-------------|-----------------|---------------|----------------|
|casual|Friday|298859|20.98|
|casual|Monday|223046|21.38|
|casual|Saturday|388725|24.47|
|casual|Sunday|314403|25.22|
|casual|Thursday|262864|18.75|
|casual|Tuesday|233246|19.54|
|casual|Wednesday|239063|18.46|
|member|Friday|504544|12.64|
|member|Monday|469069|12.12|
|member|Saturday|442174|14.15|
|member|Sunday|379331|14.14|
|member|Thursday|577549|12.19|
|member|Tuesday|541749|12.31|
|member|Wednesday|567443|12.2|


In [ ]:

--Number of Rides per Hour for Casual Riders and Members

SELECT member_casual, strftime('%H', started_at) as start_hour_of_day, COUNT(*) AS num_of_rides
FROM tripdata
GROUP BY member_casual, strftime('%H', started_at);

|member_casual|start_hour_of_day|num_of_rides|
|-------------|-----------------|------------|
|casual|00|34648|
|casual|01|22465|
|casual|02|13491|
|casual|03|7446|
|casual|04|5532|
|casual|05|10700|
|casual|06|28276|
|casual|07|50409|
|casual|08|67755|
|casual|09|66802|
|casual|10|82845|
|casual|11|105417|
|casual|12|125058|
|casual|13|130108|
|casual|14|135460|
|casual|15|151504|
|casual|16|174154|
|casual|17|190693|
|casual|18|164159|
|casual|19|121061|
|casual|20|87459|
|casual|21|73203|
|casual|22|64986|
|casual|23|46575|
|member|00|33067|
|member|01|19641|
|member|02|11238|
|member|03|7477|
|member|04|8251|
|member|05|32311|
|member|06|100500|
|member|07|186943|
|member|08|234515|
|member|09|156639|
|member|10|140380|
|member|11|166828|
|member|12|188774|
|member|13|187726|
|member|14|191604|
|member|15|234564|
|member|16|318036|
|member|17|372241|
|member|18|293092|
|member|19|206824|
|member|20|143923|
|member|21|111330|
|member|22|83141|
|member|23|52814|


In [ ]:
----Analyze ridership data by each month for members vs casual riders

SELECT member_casual, start_month, COUNT(*) as number_of_rides, ROUND(AVG(ride_length),2)  as average_duration
FROM tripdata 
GROUP BY member_casual,start_month;

|member_casual|start_month|number_of_rides|average_duration|
|-------------|-----------|---------------|----------------|
|casual|Apr|139990|21.7|
|casual|Aug|298520|22.74|
|casual|Dec|49546|15.19|
|casual|Feb|45390|18.68|
|casual|Jan|23185|14.29|
|casual|Jul|316965|23.8|
|casual|Jun|288044|22.68|
|casual|Mar|59133|16.08|
|casual|May|223304|23.15|
|casual|Nov|94577|16.62|
|casual|Oct|169954|19.67|
|casual|Sep|251598|21.9|
|member|Apr|262026|12.16|
|member|Aug|440305|13.72|
|member|Dec|163864|11.26|
|member|Feb|168544|12.17|
|member|Jan|113005|11.94|
|member|Jul|415622|13.8|
|member|Jun|398924|13.43|
|member|Mar|183565|10.84|
|member|May|351231|13.2|
|member|Nov|252101|11.47|
|member|Oct|344335|12.03|
|member|Sep|388337|13.1|


In [ ]:
-- TOP TEN START STATIONS BY USER TYPE

SELECT member_casual, start_station_name, count(*) as number_of_rides
FROM tripdata  
WHERE start_station_name <> '' AND start_station_name IS NOT NULL AND member_casual = 'member'
GROUP BY member_casual, start_station_name 
ORDER BY count(*) DESC
LIMIT 10



SELECT member_casual, start_station_name, count(*) as number_of_rides
FROM tripdata  
WHERE start_station_name <> '' AND start_station_name IS NOT NULL AND member_casual = 'casual'
GROUP BY member_casual, start_station_name 
ORDER BY count(*) DESC
LIMIT 10


|member_casual|start_station_name|number_of_rides|
|-------------|------------------|---------------|
|member|Clinton St & Washington Blvd|25604|
|member|Kingsbury St & Kinzie St|25588|
|member|Clark St & Elm St|24135|
|member|Wells St & Concord Ln|20391|
|member|Clinton St & Madison St|20191|
|member|Wells St & Elm St|19555|
|member|Broadway & Barry Ave|18413|
|member|University Ave & 57th St|18068|
|member|Loomis St & Lexington St|17590|
|member|Canal St & Adams St|16874|


|member_casual|start_station_name|number_of_rides|
|-------------|------------------|---------------|
|casual|Streeter Dr & Grand Ave|44304|
|casual|DuSable Lake Shore Dr & Monroe St|29403|
|casual|Michigan Ave & Oak St|21991|
|casual|DuSable Lake Shore Dr & North Blvd|19744|
|casual|Millennium Park|19301|
|casual|Shedd Aquarium|16980|
|casual|Theater on the Lake|15866|
|casual|Dusable Harbor|14912|
|casual|Wells St & Concord Ln|11719|
|casual|Adler Planetarium|11536|
